In [ ]:
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

In [5]:
import pandas as pd
df =pd.read_csv("/content/drive/MyDrive/HIGGS.csv", header = None)

In [ ]:
X_train = df.iloc[:10000000, 1:]
Y_train = df.iloc[:10000000, 0]
X_val = df.iloc[10000000:10500000, 1:]
Y_val = df.iloc[10000000:10500000, 0]
X_test = df.iloc[10500000:, 1:]
Y_test = df.iloc[10500000:, 0]

X_train = X_train.values
Y_train = Y_train.values
X_val = X_val.values
Y_val = Y_val.values
X_test = X_test.values
Y_test = Y_test.values

In [7]:
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
def shallow_network():

  mean = 0.0
  ki0 = tf.keras.initializers.RandomNormal(mean=mean,stddev=1)
  opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9)
  model = Sequential([
      Dense(1000, activation='tanh', input_shape=(21,), name='h0', kernel_initializer = ki0),
      Dense(1, activation='sigmoid', name='y', kernel_initializer = ki0),
  ])

  model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy'])
  model.build()
  model.summary()

  return model

In [2]:
def four_hl_network():

  mean = 0.0
  ki0 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.1)
  ki1 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki2 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki3 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki4 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.001)
  opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9)
  model = Sequential([
      Dense(300, activation='tanh', input_shape=(28,), name='h0', kernel_initializer = ki0),
      Dense(300, activation='tanh', name='h1', kernel_initializer = ki1),
      Dense(300, activation='tanh', name='h2', kernel_initializer = ki2),
      Dense(300, activation='tanh',  name='h3', kernel_initializer = ki3),
      Dense(1, activation='sigmoid', name='y', kernel_initializer = ki4),
  ])

  model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy'])
  model.build()
  model.summary()

  return model

In [8]:
def five_hl_network():

  mean = 0.0
  ki0 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.1)
  ki1 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki2 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki3 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki4 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.05)
  ki5 = tf.keras.initializers.RandomNormal(mean=mean,stddev=0.001)
  opt = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9)
  model = Sequential([
      Dense(300, activation='tanh', input_shape=(28,), name='h0', kernel_initializer = ki0),
      Dense(300, activation='tanh', name='h1', kernel_initializer = ki1),
      Dense(300, activation='tanh', name='h2', kernel_initializer = ki2),
      Dense(300, activation='tanh',  name='h3', kernel_initializer = ki3),
      Dense(300, activation='tanh',  name='h4', kernel_initializer = ki4),
      Dense(1, activation='sigmoid', name='y', kernel_initializer = ki5),
  ])

  # compile the keras model
  model.compile(loss='binary_crossentropy', optimizer =opt, metrics=['accuracy'])
  model.build()
  model.summary()

  return model

In [9]:
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np


class PlotLearning(keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [10]:
my_callbacks = [
    PlotLearning(),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/Higgs_model_checkpoints/filepath_loss.cktp', 
                                       save_weights_only=True,
                                       monitor='loss',
                                       mode='min',
                                       save_best_only=True),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/Higgs_model_checkpoints/filepath_valloss.cktp', 
                                       save_weights_only=True,
                                       monitor='val_loss',
                                       mode='min',
                                       save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', verbose=1, min_lr=1e-6)
]

In [ ]:
model = five_hl_network()

In [ ]:
model.load_weights('/content/drive/MyDrive/Higgs_model_checkpoints/filepath.cktp')

In [ ]:
print(tf.keras.backend.eval(model.optimizer.lr))

In [ ]:
tf.keras.backend.set_value(model.optimizer.lr, 0.001)

In [ ]:
hist = model.fit(X_train, Y_train,
          batch_size=128, epochs=200, callbacks=my_callbacks,
          validation_data=(X_val, Y_val))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
y_pred_keras = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y_test, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()